# Task 4: Sequential Pattern Mining

In [29]:
import pandas as pd

## Loading the new dataset

In [30]:
df = pd.read_csv('../dataset/new_customer_supermarket.csv', sep='\t', index_col=0)
df

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta,TotSale
0,539993,2011-04-01 10:00:00,1.95,13313.0,United Kingdom,22386,JUMBO BAG PINK POLKADOT,10,19.50
1,539993,2011-04-01 10:00:00,0.42,13313.0,United Kingdom,21499,BLUE POLKADOT WRAP,25,10.50
2,539993,2011-04-01 10:00:00,0.42,13313.0,United Kingdom,21498,RED RETROSPOT WRAP,25,10.50
3,539993,2011-04-01 10:00:00,2.10,13313.0,United Kingdom,22379,RECYCLING BAG RETROSPOT,5,10.50
4,539993,2011-04-01 10:00:00,1.25,13313.0,United Kingdom,20718,RED RETROSPOT SHOPPER BAG,10,12.50
...,...,...,...,...,...,...,...,...,...
363572,581587,2011-09-12 12:50:00,0.85,12680.0,France,22613,PACK OF SPACEBOY NAPKINS,12,10.20
363573,581587,2011-09-12 12:50:00,2.10,12680.0,France,22899,CHILDRENS APRON DOLLY GIRL,6,12.60
363574,581587,2011-09-12 12:50:00,4.15,12680.0,France,23254,CHILDRENS CUTLERY DOLLY GIRL,4,16.60
363575,581587,2011-09-12 12:50:00,4.15,12680.0,France,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,16.60


In [31]:
df.dtypes

BasketID             int64
BasketDate          object
Sale               float64
CustomerID         float64
CustomerCountry     object
ProdID              object
ProdDescr           object
Qta                  int64
TotSale            float64
dtype: object

In [32]:
df = df.astype({'BasketDate': 'datetime64',
                'BasketID': 'object',
                'CustomerID': 'object'})

In [33]:
df['ProdDescr'].nunique()

3678

In [34]:
import webcolors
from nltk.corpus import stopwords
from nltk import word_tokenize

stop_words = set(stopwords.words('english'))
colors = webcolors.CSS3_NAMES_TO_HEX.keys()

def normalize_descr(descr):
    return ' '.join(token for token in word_tokenize(descr.lower()) 
                    if not token in stop_words and 
                    not any(color in token for color in colors) and 
                    len(token) >= 3)

In [35]:
df['ProdDescr'] = df['ProdDescr'].apply(lemmatize_descr)
df

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta,TotSale
0,539993,2011-04-01 10:00:00,1.95,13313,United Kingdom,22386,jumbo bag polkadot,10,19.50
1,539993,2011-04-01 10:00:00,0.42,13313,United Kingdom,21499,polkadot wrap,25,10.50
2,539993,2011-04-01 10:00:00,0.42,13313,United Kingdom,21498,retrospot wrap,25,10.50
3,539993,2011-04-01 10:00:00,2.10,13313,United Kingdom,22379,recycling bag retrospot,5,10.50
4,539993,2011-04-01 10:00:00,1.25,13313,United Kingdom,20718,retrospot shopper bag,10,12.50
...,...,...,...,...,...,...,...,...,...
363572,581587,2011-09-12 12:50:00,0.85,12680,France,22613,pack spaceboy napkin,12,10.20
363573,581587,2011-09-12 12:50:00,2.10,12680,France,22899,childrens apron dolly girl,6,12.60
363574,581587,2011-09-12 12:50:00,4.15,12680,France,23254,childrens cutlery dolly girl,4,16.60
363575,581587,2011-09-12 12:50:00,4.15,12680,France,23255,childrens cutlery circus parade,4,16.60


In [36]:
df['ProdDescr'].nunique()

3259

## GSP: Apriori-based Sequential Pattern Mining

In [38]:
import copy
import itertools
from collections import defaultdict
from operator import itemgetter

In [65]:
# An event is a list of strings.
# A sequence is a list of events.
# A dataset is a list of sequences.
# Thus, a dataset is a list of lists of lists of strings.

dataset = [
    [["a"], ["a", "b", "c"], ["a", "c"], ["c"]], 
    [["a"], ["c"], ["b", "c"]], 
    [["a", "b"], ["d"], ["c"], ["b"], ["c"]], 
    [["a"], ["c"], ["b"], ["c"]]
]

In [40]:
"""
This is a simple recursive method that checks if subsequence is a subSequence of mainSequence
"""
def isSubsequence(mainSequence, subSequence):
    subSequenceClone = list(subSequence) # clone the sequence, because we will alter it
    return isSubsequenceRecursive(mainSequence, subSequenceClone) # start recursion

"""
Function for the recursive call of isSubsequence, not intended for external calls
"""
def isSubsequenceRecursive(mainSequence, subSequenceClone, start=0):
    # Check if empty: End of recursion, all itemsets have been found
    if not subSequenceClone:
        return True
    # retrieves element of the subsequence and removes is from subsequence 
    firstElem = set(subSequenceClone.pop(0))
    # Search for the first itemset...
    for i in range(start, len(mainSequence)):
        if set(mainSequence[i]).issuperset(firstElem):
            # and recurse
            return isSubsequenceRecursive(mainSequence, subSequenceClone, i + 1)
    return False

In [41]:
aSequence = [["a"], ["b", "c"], ["d"], ["a", "e"]]

In [42]:
isSubsequence(aSequence, [["a"], ["d"], ["e"]])

True

In [43]:
isSubsequence(aSequence, [["a"], ["b", "c"], ["e"]])

True

In [44]:
isSubsequence(aSequence, [["a"], ["b", "d"]])

False

In [45]:
"""
Computes the length of the sequence (sum of the length of the contained itemsets)
"""
def sequenceLength(sequence):
    return sum(len(i) for i in sequence)

In [46]:
print(sequenceLength([["a"], ["b", "c"], ["a"], ["b", "c", "d"]]))

7


In [47]:
"""
Computes the support of a sequence in a dataset
"""
def countSupport(dataset, candidateSequence):
    return sum(1 for seq in dataset if isSubsequence(seq, candidateSequence))

In [48]:
countSupport(dataset, [["b"]])

4

In [49]:
countSupport(dataset, [["a"], ["b", "c"]])

2

In [50]:
"""
Generates one candidate of length k from two candidates of length (k-1) as used in the AprioriAll algorithm
"""
def generateCandidatesForPair(cand1, cand2):
    cand1Clone = copy.deepcopy(cand1)
    cand2Clone = copy.deepcopy(cand2)
    # drop the leftmost item from cand1:
    if len(cand1[0]) == 1:
        cand1Clone.pop(0)
    else:
        cand1Clone[0] = cand1Clone[0][1:]
    # drop the rightmost item from cand2:
    if len(cand2[-1]) == 1:
        cand2Clone.pop(-1)
    else:
        cand2Clone[-1] = cand2Clone[-1][:-1]
    
    # if the result is not the same, then we dont need to join
    if not cand1Clone == cand2Clone:
        return []
    else:
        newCandidate = copy.deepcopy(cand1)
        if len(cand2[-1]) == 1:
            newCandidate.append(cand2[-1])
        else:
            newCandidate[-1].extend(cand2[-1][-1])
        return newCandidate

In [51]:
candidateA = [["a"], ["b", "c"], ["d"]]
candidateB = [["b", "c"], ["d", "e"]]
generateCandidatesForPair(candidateA, candidateB)

[['a'], ['b', 'c'], ['d', 'e']]

In [52]:
candidateA = [["a"], ["b", "c"], ["d"]]
candidateC = [["b", "c"], ["d"], ["e"]]
generateCandidatesForPair(candidateA, candidateC)

[['a'], ['b', 'c'], ['d'], ['e']]

In [53]:
candidateA = [["a"], ["b", "c"], ["d"]]
candidateD = [["a"], ["b", "c"], ["e"]]
generateCandidatesForPair(candidateA, candidateD)

[]

In [54]:
"""
Generates the set of candidates of length k from the set of frequent sequences with length (k-1)
"""
def generateCandidates(lastLevelCandidates):
    k = sequenceLength(lastLevelCandidates[0]) + 1
    if k == 2:
        flatShortCandidates = [item for sublist2 in lastLevelCandidates for sublist1 in sublist2 for item in sublist1]
        result = [[[a, b]] for a in flatShortCandidates for b in flatShortCandidates if b > a]
        result.extend([[[a], [b]] for a in flatShortCandidates for b in flatShortCandidates])
        return result
    else:
        candidates = []
        for i in range(0, len(lastLevelCandidates)):
            for j in range(0, len(lastLevelCandidates)):
                newCand = generateCandidatesForPair(lastLevelCandidates[i], lastLevelCandidates[j])
                if not newCand == []:
                    candidates.append(newCand)
        candidates.sort()
        return candidates

An example; lets assume, we know the frequent sequences of level 2:

In [55]:
lastLevelFrequentPatterns = [
    [['a', 'b']], 
    [['b', 'c']], 
    [['a'], ['b']], 
    [['a'], ['c']], 
    [['b'], ['c']], 
    [['c'], ['b']], 
    [['c'], ['c']]
]

Then we can compute the generate candidates for level 3

In [56]:
newCandidates = generateCandidates(lastLevelFrequentPatterns)
newCandidates

[[['a'], ['b'], ['c']],
 [['a'], ['b', 'c']],
 [['a'], ['c'], ['b']],
 [['a'], ['c'], ['c']],
 [['a', 'b'], ['c']],
 [['a', 'b', 'c']],
 [['b'], ['c'], ['b']],
 [['b'], ['c'], ['c']],
 [['b', 'c'], ['b']],
 [['b', 'c'], ['c']],
 [['c'], ['b'], ['c']],
 [['c'], ['b', 'c']],
 [['c'], ['c'], ['b']],
 [['c'], ['c'], ['c']]]

In [57]:
"""
Computes all direct subsequence for a given sequence.
A direct subsequence is any sequence that originates from deleting exactly one item from any event in the original sequence.
"""
def generateDirectSubsequences(sequence):
    result = []
    for i, itemset in enumerate(sequence):
        if len(itemset) == 1:
            sequenceClone = copy.deepcopy(sequence)
            sequenceClone.pop(i)
            result.append(sequenceClone)
        else:
            for j in range(len(itemset)):
                sequenceClone = copy.deepcopy(sequence)
                sequenceClone[i].pop(j)
                result.append(sequenceClone)
    return result

In [69]:
"""
Prunes the set of candidates generated for length k given all frequent sequence of level (k-1), as done in AprioriAll
"""
def pruneCandidates(candidatesLastLevel, candidatesGenerated):
    return [cand for cand in candidatesGenerated 
            if all(x in candidatesLastLevel for x in generateDirectSubsequences(cand))]

We apply this on example dataset

In [59]:
candidatesPruned = pruneCandidates(lastLevelFrequentPatterns, newCandidates)
candidatesPruned

[[['a'], ['b'], ['c']],
 [['a'], ['b', 'c']],
 [['a'], ['c'], ['b']],
 [['a'], ['c'], ['c']],
 [['a', 'b'], ['c']],
 [['b'], ['c'], ['c']],
 [['b', 'c'], ['c']],
 [['c'], ['b'], ['c']],
 [['c'], ['b', 'c']],
 [['c'], ['c'], ['b']],
 [['c'], ['c'], ['c']]]

In [60]:
minSupport = 2
candidatesCounts = [(i, countSupport(dataset, i)) for i in candidatesPruned]
resultLvl = [(i, count) for i, count in candidatesCounts if count >= minSupport]
resultLvl

[([['a'], ['b'], ['c']], 3),
 ([['a'], ['b', 'c']], 2),
 ([['a'], ['c'], ['b']], 3),
 ([['a'], ['c'], ['c']], 4),
 ([['a', 'b'], ['c']], 2),
 ([['b'], ['c'], ['c']], 2),
 ([['c'], ['b'], ['c']], 2)]

In [67]:
"""
The GSP algorithm: Apriori-based Sequential Pattern Mining. Computes the frequent sequences in a seqeunce dataset for a given minSupport.

Args:
    dataset: a list of sequences, for which the frequent (sub-)sequences are computed
    minSupport: the minimum support that makes a sequence frequent
    verbose: if true, additional information on the mining process is printed (i.e., candidates on each level)

Returns:
    A list of tuples (s, c), where s is a frequent sequence, and c is the count for that sequence
"""
def gsp(dataset, minSupport, verbose=False):
    global numberOfCountingOperations
    numberOfCountingOperations = 0
    overall = []
    itemsInDataset = sorted(set([item for sublist1 in dataset for sublist2 in sublist1 for item in sublist2]))
    singleItemSequences = [[[item]] for item in itemsInDataset]
    singleItemCounts = [(i, countSupport(dataset, i)) for i in singleItemSequences if countSupport(dataset, i) >= minSupport]
    overall.append(singleItemCounts)
    print("Result, lvl 1: " + str(overall[0]))
    k = 1
    while True:
        if not overall[k - 1]:
            break
        # 1. Candidate generation
        candidatesLastLevel = [x[0] for x in overall[k - 1]]
        candidatesGenerated = generateCandidates(candidatesLastLevel)
        # 2. Candidate pruning (using a "containsall" subsequences)
        candidatesPruned = [cand for cand in candidatesGenerated if all(x in candidatesLastLevel for x in generateDirectSubsequences(cand))]
        # 3. Candidate checking
        candidatesCounts = [(i, countSupport(dataset, i)) for i in candidatesPruned]
        resultLvl = [(i, count) for i, count in candidatesCounts if count >= minSupport]
        if verbose:
            print("Candidates generated, lvl " + str(k + 1) + ": " + str(candidatesGenerated))
            print("Candidates pruned, lvl " + str(k + 1) + ": " + str(candidatesPruned))
            print("Result, lvl " + str(k + 1) + ": " + str(resultLvl))
        overall.append(resultLvl)
        k = k + 1
    # "flatten" overall
    overall = overall[:-1]
    overall = [item for sublist in overall for item in sublist]
    return overall

In [68]:
gsp(dataset, 2, verbose=False)

Result, lvl 1: [([['a']], 4), ([['b']], 4), ([['c']], 4)]


[([['a']], 4),
 ([['b']], 4),
 ([['c']], 4),
 ([['a', 'b']], 2),
 ([['b', 'c']], 2),
 ([['a'], ['b']], 4),
 ([['a'], ['c']], 4),
 ([['b'], ['c']], 3),
 ([['c'], ['b']], 3),
 ([['c'], ['c']], 4),
 ([['a'], ['b'], ['c']], 3),
 ([['a'], ['b', 'c']], 2),
 ([['a'], ['c'], ['b']], 3),
 ([['a'], ['c'], ['c']], 4),
 ([['a', 'b'], ['c']], 2),
 ([['b'], ['c'], ['c']], 2),
 ([['c'], ['b'], ['c']], 2),
 ([['a'], ['c'], ['b'], ['c']], 2),
 ([['a', 'b'], ['c'], ['c']], 2)]